In [1]:
import sys
import os

sys.path.append('source/')
sys.path.append('source/informational_states')


In [2]:
import numpy as np 
import matplotlib.pyplot as plt
plt.rcParams["animation.html"] = "jshtml"
from IPython.display import HTML
import matplotlib.animation as animation

%matplotlib inline

In [3]:
#greg tools
from infoenginessims.api import *
from infoenginessims.integrators import rkdeterm_eulerstoch
from infoenginessims.dynamics import langevin_underdamped, langevin_overdamped
from infoenginessims import simulation
from infoenginessims.simprocedures import basic_simprocedures as sp
from infoenginessims.simprocedures import running_measurements as rp
from infoenginessims import analysis
import infoenginessims.analysis.running_quantities
import infoenginessims.analysis.hists_1D

In [4]:
#kyle tools
import kyle_tools as kt
from protocol_designer import potentials, protocol, system
from measure import MeasurementDevice, Measurement, TrajectoryEnsemble



ModuleNotFoundError: No module named 'measure'

In [ ]:

#params 1:
kT = 4.1*1.38E-23
C = 400E-12
R = 371
L = 10E-10

'''
#params 2:
kT = 6.9E-24
C = 530E-15
R = 2.1
L = 140E-12
'''

#these are some relevant dimensionful scales: alpha are the natural units for the JJ fluxes and U_0 is the natural scale for the potential
alpha = 2.07E-15 / (2*np.pi)
U_0 = alpha**2 / L

#these are important dimensionless quantities, given that m is measured in units of C, lambda in units of 1/R, temperature in units of U_0
m_prime = np.array((2, 1/2))
lambda_prime = np.array((2, 1/2))
kT_prime = kT/U_0


### First few cells are to set up the "system": the potential and the time dependent signal sent to its parameters

In [ ]:

#some sample systems for the bit flip, defined in the fq_systems.py file. You can use that file as a model for making new protocols:

from fq_systems import fq_pot, flip_on, flip_off, flip_prot 

#equilibrating
eq_sys = system.System(fq_pot.trivial_protocol(), fq_pot)

#starts in EQ, holds the flip potential indefinitely
eternal_flip_sys = system.System(flip_on, fq_pot)

#starts in flip potential and then relaxes to EQ
diagnostic_flip_sys = system.System(flip_off, fq_pot)

#full flip, start in EQ and then end in EQ again
flip_sys = system.System(flip_prot, fq_pot)

In [ ]:
#this cell defines what system you want to simulate and how many trials to run. generally you dont need alot of trials unless you are going for statistics
N=300
system= eq_sys
#system.potential.scale = 1


In [ ]:
#initialize the state in a rough EQ distribution, not super accurate.
initial_state = eq_sys.eq_state(N, t=0, beta=1/(kT_prime), M= m_prime, manual_domain=[[-4,-3],[4,1]], axes=[1,2])

### Next few cells are visualization checks that your system is set up how you want

In [ ]:
#this cell is for checking that the initial state is what you want
nbins= 20
x_hist = np.histogram(initial_state[:,0,0], bins=nbins)
y_hist = np.histogram(initial_state[:,1,0], bins=nbins)
vx_hist = np.histogram(initial_state[:,0,1], bins=nbins)
vy_hist = np.histogram(initial_state[:,1,1], bins=nbins)
#change the type of histogram to look at different coordinates
analysis.hists_1D.plot_hist(x_hist);

In [ ]:
#gives a snapshot of the potential at some time in some domain
system.show_potential(0, manual_domain=[[-4,-4],[4,-1]], surface=False)

In [ ]:
%%capture

#next two cells make an animation of the potential and how it changes over the protocol
sysanim=system.animate_protocol(surface=False, manual_domain=[[-4,-7],[4,0]], n_contours=20)
HTML(sysanim.to_jshtml(fps=8))


In [ ]:
sysanim

### Now we set up the simulation parameters

In [ ]:
# this sets up our simulation to do langevin dynamics.
# if you want to change the temperature or damping by some amount, you can change the scale factors in this cell
# probably dont want to change anythign else in here though

#important to note that chaning the temperature here will not change the temperature used to generate the EQ distribution,
# this is in case you want to test equilibration at a different temperature than computation.


#note: time is scaled in units of sqrt(LC)
gamma = (lambda_prime/m_prime) * np.sqrt(L*C) / (R*C) 
theta = 1/m_prime
eta = (L/(R**2 * C))**(1/4) * np.sqrt(kT_prime*lambda_prime) / m_prime        
 

damping_scale = 1
temp_scale = 1

gamma = np.multiply(gamma, damping_scale)
eta = np.multiply(eta, np.sqrt(damping_scale*temp_scale))

dynamic = langevin_underdamped.LangevinUnderdamped(theta, gamma, eta, system.get_external_force)

integrator = rkdeterm_eulerstoch.RKDetermEulerStoch(dynamic)

In [ ]:
#dont change this cell unless you take a look at how the procedures work, this should be fine for most use cases

procedures = [
              sp.ReturnFinalState(),
              sp.MeasureAllState(trial_request=slice(0, 1000)),  
              rp.MeasureAllValue(rp.get_dW, 'all_W'),
              rp.MeasureFinalValue(rp.get_dW, 'final_W'),
             ]

In [ ]:
# here is where you choose the number of steps to simulate and how long to run the sim for.
# note that if your time is longer than the protocol time, the potential will just sit at its final value.

nsteps = 10_000

total_time = 10*(system.protocol.t_f-system.protocol.t_i)

dt = total_time / nsteps

sim = simulation.Simulation(integrator.update_state, procedures, nsteps, dt,
                            initial_state)

sim.system = system

### This is running the actual sim

In [ ]:
%%time

sim.output = sim.run(verbose=True)

In [ ]:
# this is assinging variables to the different sim outputs
all_state = sim.output.all_state['states']
all_W = sim.output.all_W
final_W = sim.output.final_W
final_state = sim.output.final_state


### After running the sim, there are plenty of analysis tools

In [ ]:
#plotting the trajectories along a particular axis

end_plot_time = total_time #* 1 / 100
trial_indices = np.s_[:300]


analysis.running_quantities.plot_running_quantity(all_state[trial_indices,:,0,0],
                                                  final_time=total_time,
                                                  end_plot_time=end_plot_time, title='x v t')

analysis.running_quantities.plot_running_quantity(all_state[trial_indices,:,1,0],
                                                  final_time=total_time,
                                                  end_plot_time=end_plot_time, title='y v t')



In [ ]:
#plotting potential energy

times = np.linspace(0, total_time, nsteps)

potentials = [system.get_potential(all_state[:,i,...], item) for i,item in enumerate(times)]

fig, ax = plt.subplots(figsize=(15,5))

ax.plot(times, potentials);


In [ ]:
#you can check the total energy here, good way to check that you are close enough to the EQ distribution
total_energy = np.array([system.get_energy(all_state[:,i,...], item, mass=m_prime) for i,item in enumerate(times)])

fig, ax = plt.subplots(figsize=(15,5))

ax.plot(times, np.mean(total_energy, axis=1));

In [ ]:
%%capture

#these cells make am animation of a 2D slice of phase space trajectories. You can plot velocities by changing the zero in all_state[...,0] to a 1
ani_exp = kt.animate_sim(all_state[...,0], total_time, color_by_state=True, key_state=None)
HTML(ani_exp.to_jshtml(fps=20))

In [ ]:
ani_exp

In [ ]:
analysis.running_quantities.plot_running_quantity(all_W[:1000],
                                                  final_time=total_time,
                                                  end_plot_time=total_time)

In [ ]:
#this will show you a histogram of the net work, with the mean and +- sigma marked

final_W_hist = np.histogram(final_W, bins=50)
fig, ax = analysis.hists_1D.plot_hist(final_W_hist, log=True)
m=final_W.mean()
s=final_W.std()
print(m)
ax.axvline(m, color='k')
ax.axvline(m-s, color='k')
ax.axvline(m+s, color='k')